In [1]:
import pandas
import data_process
import tools
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm



In [2]:
class rnn(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, device):
        super(rnn, self).__init__()
        self.device = device
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 2)  # 假设是二分类任务
    
    def forward(self, x):
        x = self.embedding(x)  # 获取词嵌入
        out, _ = self.rnn(x)  # RNN 前向传播
        out = out[:, -1, :]  # 取最后一个时间步的输出
        out = self.fc(out)  # 输出层
        return out

In [3]:

class OptimizedRNN(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, device):
        super(OptimizedRNN, self).__init__()
        self.device = device
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True)  # 双向LSTM
        self.dropout = nn.Dropout(0.5)  # Dropout正则化
        self.fc = nn.Linear(hidden_size * 2, 2)  # 双向LSTM输出维度需要乘2
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.embedding(x)  # 获取词嵌入
        out, _ = self.rnn(x)  # LSTM前向传播
        out = out[:, -1, :]  # 取最后一个时间步的输出
        out = self.dropout(out)  # 应用Dropout
        out = self.fc(out)  # 输出层
        out = self.softmax(out)  # 进行Softmax归一化
        return out



In [4]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx, device):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, 
                           bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        # text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))  # embedded = [sent len, batch size, emb dim]
        
        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        # Unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # Concatenate the final forward and backward hidden states and apply dropout
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))  # [batch size, hid dim * num directions]
            
        return self.fc(hidden)  # Output of shape [batch size, output dim]


包装数据

In [5]:
import torch
from torch.nn.utils.rnn import pad_sequence

# 自定义 collate_fn
def collate_fn(batch):
    texts, labels = zip(*batch)
    
    # 文本 padding，先转换为 Tensor
    text_lengths = torch.tensor([len(text) for text in texts])
    
    # 将文本进行 padding
    padded_texts = pad_sequence([torch.tensor(text) for text in texts], padding_value=0, batch_first=True)
    
    # 将标签转换为 Tensor
    labels = torch.tensor(labels)

    # 确保 text_lengths 是 Long 类型，并且在 CPU 上
    text_lengths = text_lengths.to(torch.long).cpu()

    return padded_texts, text_lengths, labels


In [6]:
data  = pandas.read_csv('./motionClassify.csv')
vocab = data_process.gen_vocab(data)
data_train  =  data_process.gen_dataset(data[:40000],vocab)
data_test = data_process.gen_dataset(data[40000:],vocab)
Batch_size = 64
train_iter = torch.utils.data.DataLoader(data_train, batch_size=Batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(data_test,Batch_size,shuffle=True)

In [7]:
# def get_pad_index(vocab):
#     # 如果 vocab 中没有 <pad>，手动添加一个索引
#     if '<pad>' not in vocab.token_to_idx:
#         # 获取当前词汇表的大小作为 <pad> 的索引
#         PAD_IDX = len(vocab.token_to_idx)
        
#         # 添加 <pad> 到 token_to_idx 和 idx_to_token
#         vocab.token_to_idx['<pad>'] = PAD_IDX
#         vocab.idx_to_token[PAD_IDX] = '<pad>'
        
#         # 重新生成 idx_to_token 字典，确保同步更新
#         vocab.idx_to_token = {idx: token for token, idx in vocab.token_to_idx.items()}
#     else:
#         PAD_IDX = vocab.token_to_idx['<pad>']  # 如果已有 <pad>，则直接获取其索引
    
#     print(f'PAD index: {PAD_IDX}')
#     print(f'Updated Vocab Size (token_to_idx): {len(vocab.token_to_idx)}')  # 打印 token_to_idx 的大小
#     print(f'Updated Vocab Size (idx_to_token): {len(vocab.idx_to_token)}')  # 打印 idx_to_token 的大小
#     return PAD_IDX


In [8]:
lr=0.05
# PAD_IDX = get_pad_index(vocab)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda:0')
#print(f"PAD Index: {PAD_IDX}, Vocab Size: {len(vocab.token_to_idx)}")

net1 = rnn(num_embeddings=len(vocab),embedding_dim=256,hidden_size=256,device=device)
net2 = OptimizedRNN(num_embeddings=len(vocab), embedding_dim=256, hidden_size=256, device=device)
net3=RNN(vocab_size=len(vocab),embedding_dim=100,hidden_dim=256,output_dim=1,n_layers=2,bidirectional=True,dropout=0.5,pad_idx=vocab.pad,device=device)
optimizer1 = torch.optim.SGD(net1.parameters(),lr)
optimizer2 = torch.optim.SGD(net2.parameters(),lr)
optimizer = torch.optim.Adam(net3.parameters())



In [9]:
tools.train(net1,train_iter,device,optimizer1,criterion)

 18%|█▊        | 113/625 [00:01<00:07, 72.96it/s]

batch100,loss = 0.6793975830078125


 34%|███▎      | 210/625 [00:03<00:05, 73.18it/s]

batch200,loss = 0.7014930248260498


 50%|█████     | 315/625 [00:04<00:04, 73.86it/s]

...

 82%|████████▏ | 515/625 [00:07<00:01, 73.23it/s]

batch500,loss = 0.6945629715919495


 98%|█████████▊| 612/625 [00:08<00:00, 73.49it/s]

batch600,loss = 0.7008679509162903


100%|██████████| 625/625 [00:08<00:00, 72.27it/s]


In [10]:
tools.test(net1,test_iter,device)

100%|██████████| 157/157 [00:00<00:00, 178.12it/s]

accuracy = 0.4992999732494354


准确率50%，接近自然概率，训练没有效果

In [11]:
tools.train(net2,train_iter,device,optimizer2,criterion)

 17%|█▋        | 104/625 [00:04<00:22, 23.11it/s]

batch100,loss = 0.6917610168457031


 32%|███▏      | 203/625 [00:08<00:18, 22.93it/s]

batch200,loss = 0.6940086483955383


 48%|████▊     | 302/625 [00:13<00:14, 22.65it/s]

...

 80%|████████  | 503/625 [00:21<00:05, 22.89it/s]

batch500,loss = 0.6940557956695557


 96%|█████████▋| 602/625 [00:26<00:01, 22.68it/s]

batch600,loss = 0.7104886770248413


100%|██████████| 625/625 [00:27<00:00, 22.92it/s]


In [12]:
tools.test(net2,test_iter,device)

100%|██████████| 157/157 [00:02<00:00, 54.29it/s]

accuracy = 0.4992999732494354


准确率50%，没有训练效果

In [13]:
tools.train(net3,train_iter,device,optimizer,criterion)

  0%|          | 0/625 [00:00<?, ?it/s]

TypeError: RNN.forward() missing 1 required positional argument: 'text_lengths'

In [ ]:
tools.test(net3,test_iter,device)